<a href="https://colab.research.google.com/github/ammaradam-skymind/darknet/blob/master/YOLOv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv4 for Custom Dataset

Reference: [Link](https://medium.com/analytics-vidhya/implementing-yolov4-to-detect-custom-objects-using-google-colab-6691c98b15ff)

![Comparison of YOLOv4 to SOTA object detectors](https://miro.medium.com/max/1400/1*yLe9jJTMmdYPT_TZSDdb-w.jpeg)

## Environment Setup



### Check GPU
- Often include Nvidia K80s, T4s, P4s and P100s

In [0]:
!nvidia-smi

Fri Jun  5 09:17:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# Check CUDA version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


### To avoid DISCONNECT in Google Colab:
- Press `Ctrl + Shift + i` in browser to open console
- In console, paste following code:
```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)
```



### Import libraries

In [0]:
import os
import random
import xml.etree.ElementTree as ET
import cv2
import matplotlib.pyplot as plt

%matplotlib inline
!rm -fr darknet

## Darknet Build
- Using open-source neural network framework Darknet
- Download and install

In [0]:
# !git clone https://github.com/AlexeyAB/darknet/
!git clone https://github.com/ammaradam-skymind/darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 13576, done.
remote: Total 13576 (delta 0), reused 0 (delta 0), pack-reused 13576
Receiving objects: 100% (13576/13576), 12.22 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (9257/9257), done.


### Installation
- Set GPU, CUDNN, OPENCV to 1

In [0]:
% cd darknet

/content/darknet


In [0]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
# !apt update
# !apt-get install libopencv-dev

### Compile & configure
- Compile project after editing Makefile
- Download pre-trained weights of YOLOv4

In [0]:
!make &> compile.log

In [0]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [0]:
def predictImage(imageDir):
  os.system("cd /content/darknet && ./darknet detect cfg/yolov4.cfg yolov4.weights {}".format(imageDir))
  image = cv2.imread("/content/darknet/predictions.jpg")
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf()
  fig.set_size_inches(15, 8)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [0]:
predictImage("data/person.jpg")

## Configurations
- Based on your requirement select a YOLOv4 config file. I selected yolov4-custom.cfg, copy the contents of cfg/yolov4-custom.cfg to a new file cfg/yolo-obj.cfg. Adjust the parameters like batch, subdivisions, steps, max_batches accordingly. For more info on parameters refer to this [link](https://github.com/AlexeyAB/darknet/wiki/CFG-Parameters-in-the-%5Bnet%5D-section).

- **Tip:** 
Set max_batches value a minimum of 2000 classes and steps to 80% - 90% of max_batches. Also the width, height parameters should be a multiple of 32 (I have used 416x416).

- Update the classes parameter to the number of objects in the 3 yolo layers in the yolo-obj.cfg file in lines 970, 1058, 1146 to 2, since we have only 2 classes (mask, no_mask). Now similarly, update the filters parameter to filters=(classes + 5) x 3 in the 3 convolutional layers before each yolo layer. In this case classes = 2 so, set the filters to 21 in the lines 963, 1051, 1139 (Don’t write filters = (classes + 5) x 3 in the .cfg file)


In [0]:
% cd '/content/darknet/build/darknet/x64/cfg'

!cp 'yolov4-custom.cfg' 'yolo-obj.cfg'

!sed -i 's/batch=64/batch=64/g' yolo-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=8/g' yolo-obj.cfg     #8
# !sed -i 's/width=608/width=160/g' yolo-obj.cfg
# !sed -i 's/height=608/height=160/g' yolo-obj.cfg
!sed -i 's/width=608/width=256/g' yolo-obj.cfg
!sed -i 's/height=608/height=256/g' yolo-obj.cfg

!sed -i 's/max_batches = 500500/max_batches = 6000/g' yolo-obj.cfg
# !sed -i 's/max_batches = 500500/max_batches = 20000/g' yolo-obj.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/g' yolo-obj.cfg
# !sed -i 's/steps=400000,450000/steps=16000,18000/g' yolo-obj.cfg

!sed -i 's/classes=80/classes=3/g' yolo-obj.cfg
!sed -i 's/filters=255/filters=24/g' yolo-obj.cfg     # (no of class + 5) * 3


/content/darknet/build/darknet/x64/cfg


-  Create `obj.names` in the directory `build\darknet\x64\data\` with class names
- Create *train.txt* and *test.txt* files in the directory `build\darknet\x64\data\` using following code:

In [0]:
obj_class = [
             'dueDate', 
             'invDate', 
             'invNo', 
            #  'item', 
            #  'itemDesc', 
            #  'itemQty', 
            #  'suppAddr', 
            #  'suppName', 
            #  'totalPrice', 
            #  'unitPrice'
            ]

In [0]:
% cd '/content/darknet/build/darknet/x64/data/'

with open('obj.names', 'w') as f:
    for item in obj_class:
        f.write("%s\n" % item)

/content/darknet/build/darknet/x64/data


In [0]:
DATASET_DRIVEID = '1mr0WFbBUQLW3cpSMK-3-tLrYbEcCA5eV'   # training images
DATASET_DIR = '/content/darknet/build/darknet/x64/data/obj'
DATASET_ZIP_DIR = '/content/darknet/build/darknet/x64/data'

In [0]:
from google.colab import drive
from google.colab import files
import shutil

DRIVE_DATASET = '/content/gdrive/My Drive/OCR Object Detection/obj.zip'

drive.mount('/content/gdrive')
shutil.copy(DRIVE_DATASET, DATASET_ZIP_DIR + '/obj.zip')
print('Download completed!')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Download completed!


In [0]:
import zipfile

file_name = DATASET_ZIP_DIR + '/obj.zip'

os.makedirs(DATASET_ZIP_DIR, exist_ok=True)
with zipfile.ZipFile(file_name, 'r') as zip_ref:
  zip_ref.extractall(DATASET_ZIP_DIR)
os.remove(file_name)
print('Extract completed!')

Extract completed!


In [0]:
for xml_file in [f for f in os.listdir(DATASET_DIR) if f.endswith(".xml")]:
  tree = ET.parse(os.path.join(DATASET_DIR, xml_file))
  root = tree.getroot()

  obj_list = []

  for elem in root:

    if elem.tag == 'size':
      for subelem in elem:
        if subelem.tag == 'width':
          img_width = subelem.text
        if subelem.tag == 'height':
          img_height = subelem.text

    if elem.tag == 'object':
      obj_name = None
      obj_classId = None
      for subelem in elem:
        if subelem.tag == 'name':
          # ONLY IF TRAINING SPECIFIC CLASSES
          if subelem.text == 'dueDate' or subelem.text == 'invDate' or subelem.text == 'invNo':
            obj_name = subelem.text
            obj_classId = obj_class.index(obj_name)
        if subelem.tag == 'bndbox':
          for subsubelem in subelem:
            if subsubelem.tag == 'xmin':
              xmin = float("{:.6f}".format(int(subsubelem.text)/int(img_width)))
            if subsubelem.tag == 'xmax':
              xmax = float("{:.6f}".format(int(subsubelem.text)/int(img_width)))
            if subsubelem.tag == 'ymin':
              ymin = float("{:.6f}".format(int(subsubelem.text)/int(img_height)))
            if subsubelem.tag == 'ymax':
              ymax = float("{:.6f}".format(int(subsubelem.text)/int(img_height)))
      
      if obj_classId is not None:
        obj_elem = "{} {} {} {} {}".format(obj_classId, xmin, ymin, xmax, ymax)
        obj_list.append(obj_elem)
  
  text_file = xml_file.replace(".xml", ".txt")

  # print(text_file)
  # print(obj_list)
  # print('\n'.join(obj_list))

  f=open(os.path.join(DATASET_DIR, text_file),'w')
  s1='\n'.join(obj_list)
  f.write(s1)
  f.close()

In [0]:
# Replace with path to your images
path = '/content/darknet/build/darknet/x64/data/obj/'

images = []
for i in [f for f in os.listdir(path) if f.endswith(".jpg")]:
    temp = path+i
    images.append(temp)
    # print(temp)

# train and test split... adjust it if necessary
trainlen = round(len(images)*.80)
testlen = round(len(images)*.10)
validlen = round(len(images)*.10)

print("-- Dataset Size --\n Total: {}\n Train: {}\n Test: {}\n Validate: {}".format(trainlen + testlen, trainlen, testlen, validlen))

random.shuffle(images)
valid = images[:validlen]
test = images[validlen:validlen+testlen]
train = images[validlen+testlen:]

text_path = '/content/darknet/build/darknet/x64/data/'

with open(text_path + 'train.txt', 'w') as f:
    for item in train:
        f.write("%s\n" % item)

with open(text_path + 'test.txt', 'w') as f:
    for item in test:
        f.write("%s\n" % item)

with open(text_path + 'valid.txt', 'w') as f:
    for item in valid:
        f.write("%s\n" % item)

-- Dataset Size --
 Total: 521
 Train: 463
 Test: 58
 Validate: 58


- Put all the image files(.jpg) and label files(.txt) in the same directory: `build\darknet\x64\data\obj`
- Make `obj.data` file in directory `build\darknet\x64\data\` which should contain below: (edit where necessary)
```
classes= 10
train = build/darknet/x64/data/train.txt
valid = build/darknet/x64/data/test.txt
names = build/darknet/x64/data/obj.names
backup = build/darknet/x64/backup/
```
- Then, download pre-trained weights for convolutional layers [here](https://drive.google.com/open?id=1JKF-bdIklxOOVy-2Cr5qdvjgGpmGfcbp) (depends on configuration used)
- Put in `build\darknet\x64`

In [0]:
% cd '/content/darknet/build/darknet/x64/data/'

with open('obj.data', 'w') as f:
  f.write("%s\n" % 'classes= 3')
  # f.write("%s\n" % 'classes= 10')
  f.write("%s\n" % 'train = build/darknet/x64/data/train.txt')
  f.write("%s\n" % 'valid = build/darknet/x64/data/test.txt')
  f.write("%s\n" % 'names = build/darknet/x64/data/obj.names')
  f.write("%s" % 'backup = build/darknet/x64/backup/')

/content/darknet/build/darknet/x64/data


In [0]:
# % cd '/content/darknet/build/darknet/x64'

# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [0]:
DRIVE_WEIGHTS = '/content/gdrive/My Drive/HLA/Models/YOLOv4/Exp1/yolov4-obj_best_61.weights'
COLAB_WEIGHTS = '/content/darknet/build/darknet/x64/yolov4-obj_best_61.weights'

# drive.mount('/content/gdrive')
shutil.copy(DRIVE_WEIGHTS, COLAB_WEIGHTS)
print('Download completed!')

Download completed!


## Training

In [0]:
% cd '/content/darknet'

# !./darknet detector train build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/yolov4.conv.137 -dont_show
!./darknet detector train build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/yolov4-obj_best_61.weights -map -dont_show -clear

Streaming output truncated to the last 5000 lines.
 1248: 2.424006, 1.853055 avg loss, 0.001000 rate, 4.842057 seconds, 79872 images, 11.847959 hours left
Loaded: 4.247679 seconds - performance bottleneck on CPU or Disk HDD/SSD

 (next mAP calculation at 1300 iterations) 
 Last accuracy mAP@0.5 = 28.84 %, best = 68.87 % 
 1249: 2.466992, 1.914449 avg loss, 0.001000 rate, 4.686728 seconds, 79936 images, 11.838820 hours left
Loaded: 2.823162 seconds - performance bottleneck on CPU or Disk HDD/SSD

 (next mAP calculation at 1300 iterations) 
 Last accuracy mAP@0.5 = 28.84 %, best = 68.87 % 
 1250: 1.856129, 1.908617 avg loss, 0.001000 rate, 4.687592 seconds, 80000 images, 11.838341 hours left
Resizing, random_coef = 1.40 

 320 x 320 
 try to allocate additional workspace_size = 29.49 MB 
 CUDA allocate done! 
Loaded: 5.404824 seconds - performance bottleneck on CPU or Disk HDD/SSD

 (next mAP calculation at 1300 iterations) 
 Last accuracy mAP@0.5 = 28.84 %, best = 68.87 % 
 1251: 2.3534

In [0]:
COLAB_MODEL = '/content/darknet/build/darknet/x64/backup/yolo-5000.weights'
DRIVE_MODEL = '/content/gdrive/My Drive/OCR Object Detection/yolo-obj_5000.weights'

shutil.copy(COLAB_MODEL, DRIVE_MODEL)

- Open [URL](http://ip-address:8090) to display mAP

In [0]:
# To train further
!./darknet detector train build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-5000.weights -dont_show -map

In [0]:
COLAB_MODEL = '/content/darknet/build/darknet/x64/backup/yolo-5000.weights'
DRIVE_MODEL = '/content/gdrive/My Drive/OCR Object Detection/yolo-obj_10000.weights'

shutil.copy(COLAB_MODEL, DRIVE_MODEL)

## Detection

### Test on Video
- To perform object detection on a video
- Saved as `result.avi`

In [0]:
# ./darknet detector demo build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_1000.weights build/darknet/x64/data/peralta_holt_mumps.mp4 -out_filename result.avi -ext_output -dont_show

### Test on Image
- To perform object detection on an image
- Saved as `test.jpg`

In [0]:
# !./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_1000.weights test.jpg -dont_show
!./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_final.weights build/darknet/x64/data/obj/Invoice_1.jpg -dont_show

In [0]:
def predictImage(imageDir):
  os.system("cd /content/darknet && ./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_final.weights {}".format(imageDir))
  image = cv2.imread("/content/darknet/predictions.jpg")
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf()
  fig.set_size_inches(15, 8)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [0]:
predictImage("/content/darknet/build/darknet/x64/data/obj/Invoice_1.jpg")

### Test on Real-time Feed
- To perform object detection on a real-time video

In [0]:
# !./darknet detector demo build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_1000.weights -c 0

### Test on Image using Weights from GDrive

In [0]:
from google.colab import drive
from google.colab import files
import shutil

DRIVE_WEIGHTS = '/content/gdrive/My Drive/HLA/Models/YOLOv4/Exp1/yolov4-obj_best.weights'
COLAB_WEIGHTS = '/content/darknet/build/darknet/x64/backup/yolov4-obj_best.weights'

# drive.mount('/content/gdrive')
shutil.copy(DRIVE_WEIGHTS, COLAB_WEIGHTS)
print('Download completed!')

In [0]:
% cd '/content/darknet'

!./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolov4-obj.cfg build/darknet/x64/backup/yolov4-obj_best.weights build/darknet/x64/data/obj/Invoice_118_118.jpg -dont_show

In [0]:
image = cv2.imread("/content/darknet/predictions.jpg")
height, width = image.shape[:2]
resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
# resized_image = cv2.resize(image,(width, height), interpolation = cv2.INTER_CUBIC)
fig = plt.gcf()
fig.set_size_inches(30, 16)
plt.axis("off")
plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
plt.show()

In [0]:
import matplotlib.patches as patches

for xml_file in [f for f in os.listdir(DATASET_DIR) if f.endswith(".xml")][:1]:
  xml_file = "Invoice_118_118.xml"

  tree = ET.parse(os.path.join(DATASET_DIR, xml_file))
  root = tree.getroot()

  obj_list = []

  image_file = xml_file.replace(".xml", ".jpg")
  print('/content/darknet/build/darknet/x64/data/obj/' + image_file)

  image = cv2.imread("/content/darknet/build/darknet/x64/data/obj/" + image_file)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(width, height), interpolation = cv2.INTER_CUBIC)
  fig,ax = plt.subplots(1)
  fig.set_size_inches(30, 16)
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))

  for elem in root:

    if elem.tag == 'size':
      for subelem in elem:
        if subelem.tag == 'width':
          img_width = subelem.text
        if subelem.tag == 'height':
          img_height = subelem.text

    if elem.tag == 'object':
      obj_name = None
      obj_classId = None
      for subelem in elem:
        if subelem.tag == 'name':
          # ONLY IF TRAINING SPECIFIC CLASSES
          # if subelem.text == 'totalPrice' or subelem.text == 'unitPrice':
            obj_name = subelem.text
            obj_classId = obj_class.index(obj_name)
        if subelem.tag == 'bndbox':
          for subsubelem in subelem:
            if subsubelem.tag == 'xmin':
              xmin = float("{:.6f}".format(int(subsubelem.text)/int(img_width)))
            if subsubelem.tag == 'xmax':
              xmax = float("{:.6f}".format(int(subsubelem.text)/int(img_width)))
            if subsubelem.tag == 'ymin':
              ymin = float("{:.6f}".format(int(subsubelem.text)/int(img_height)))
            if subsubelem.tag == 'ymax':
              ymax = float("{:.6f}".format(int(subsubelem.text)/int(img_height)))
      
      if obj_classId is not None:
        obj_elem = "{} {} {} {} {}".format(obj_classId, xmin, ymin, xmax, ymax)
        obj_list.append(obj_elem)

    print("{} {} {} {} {}".format(obj_name, xmin*width, ymin*height, xmax*width, ymax*height))

    # Create a Rectangle patch
    rect = patches.Rectangle((xmin*width,ymin*height),(xmax-xmin)*width,(ymax-ymin)*height,linewidth=1,edgecolor='r',facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

    plt.text(xmin*width,ymin*height,obj_name,fontsize=12)
  
  text_file = xml_file.replace(".xml", ".txt")

  plt.show()

# print('/content/darknet/build/darknet/x64/data/obj/' + image_file)
# print("{} {} {} {} {}".format(obj_name, xmin*width, ymin*height, xmax*width, ymax*height))

# image = cv2.imread("/content/darknet/build/darknet/x64/data/obj/" + image_file)
# height, width = image.shape[:2]
# resized_image = cv2.resize(image,(width, height), interpolation = cv2.INTER_CUBIC)
# # fig = plt.gcf()
# fig,ax = plt.subplots(1)
# fig.set_size_inches(30, 16)
# # plt.axis("off")
# plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))

# # Create a Rectangle patch
# rect = patches.Rectangle((xmin*width,ymin*height),(xmax-xmin)*width,(ymax-ymin)*height,linewidth=1,edgecolor='r',facecolor='none')

# # Add the patch to the Axes
# ax.add_patch(rect)

# plt.show()

  # print(text_file)
  # print(obj_list)
  # print('\n'.join(obj_list))

  # f=open(os.path.join(DATASET_DIR, text_file),'w')
  # s1='\n'.join(obj_list)
  # f.write(s1)
  # f.close()

#### To calculate anchors:

In [0]:
!./darknet detector calc_anchors build/darknet/x64/data/obj.data -num_of_clusters 9 -width 256 -height 256

#### To check accuracy mAP@IoU=50:

In [0]:
!./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolov4-obj.cfg build/darknet/x64/backup/yolov4-obj_best.weights

#### To check accuracy mAP@IoU=75:

In [0]:
!./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolov4-obj.cfg build/darknet/x64/backup/yolov4-obj_best.weights -iou_thresh 0.75

- To process a list of images data/train.txt and save results of detection to result.json file use:

`darknet.exe detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -ext_output -dont_show -out result.json < data/train.txt`

- To process a list of images data/train.txt and save results of detection to result.txt use:

`darknet.exe detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show -ext_output < data/train.txt > result.txt`